In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Distrito Federal - Produção de Cimento (t)', axis=1)
data

,Ano,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao,Distrito Federal - Consumo de Cimento (t)
0,2008,1.257796e+08,3.618753e+06,44.111562,1.114321e+08,0.831284,0.850754,8.042711,5.483335e+08,2.546644e+09,...,26.827393,887.117978,2116.747379,21.422126,14.146668,66.792192,2.378486,0.847880,0.795527,909.713000
1,2009,1.266042e+08,3.629545e+06,44.141542,1.114727e+08,0.831790,0.851939,8.070885,5.302764e+08,2.679945e+09,...,26.876348,886.879495,1766.115242,21.441397,14.149368,66.806007,2.417640,0.846467,0.797514,991.263000
2,2010,1.296005e+08,3.625996e+06,44.116626,1.120321e+08,0.832181,0.852532,8.035310,5.046425e+08,2.891930e+09,...,26.495387,886.771930,1650.667839,21.298714,13.834925,66.431493,2.459251,0.845301,0.799450,1167.604000
3,2011,1.323025e+08,3.587906e+06,44.022754,1.124103e+08,0.832459,0.852542,8.024653,4.854622e+08,3.091988e+09,...,26.497614,886.883230,1564.062453,21.274581,13.773402,66.359941,2.445634,0.844378,0.801325,1277.890000
4,2012,1.348108e+08,3.550906e+06,43.926397,1.127652e+08,0.832380,0.851897,10.763245,4.867661e+08,3.189506e+09,...,27.337584,886.989319,1529.804524,21.318450,13.817656,66.310747,2.408201,0.843623,0.802661,1261.667000
5,2013,1.372002e+08,3.512366e+06,43.880001,1.131945e+08,0.832995,0.850805,11.341796,5.225848e+08,3.093872e+09,...,27.062026,887.169899,1507.055019,21.353309,13.761380,66.172806,2.347696,0.843425,0.805550,1152.069000
6,2014,1.392692e+08,3.476698e+06,43.883670,1.135587e+08,0.832661,0.849205,11.799788,5.676413e+08,2.779340e+09,...,26.037094,887.401648,1507.000310,21.640954,13.621200,64.854756,2.309168,0.843044,0.806512,979.998857
7,2015,1.407164e+08,3.438025e+06,43.849398,1.137031e+08,0.832383,0.847056,12.093705,5.179392e+08,2.561731e+09,...,25.992461,887.544811,1513.172948,21.797552,13.512417,63.666484,2.318908,0.842881,0.807805,811.194686
8,2016,1.416229e+08,3.431106e+06,43.828558,1.137585e+08,0.834584,0.854633,12.238790,4.524144e+08,2.516859e+09,...,27.482499,887.589041,1508.330179,21.640241,13.367470,63.456555,2.339247,0.836089,0.813684,638.818457
9,2017,1.417351e+08,3.426736e+06,43.802546,1.136558e+08,0.833455,0.851963,12.085632,4.213689e+08,2.497710e+09,...,27.991365,887.652320,1501.483221,21.558960,13.478044,64.168678,2.297251,0.836097,0.812928,584.646000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,1.257796e+08,3.618753e+06,44.111562,1.114321e+08,0.831284,0.850754,8.042711,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,26.827393,887.117978,2116.747379,21.422126,14.146668,66.792192,2.378486,0.847880,0.795527
1,1.266042e+08,3.629545e+06,44.141542,1.114727e+08,0.831790,0.851939,8.070885,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,26.876348,886.879495,1766.115242,21.441397,14.149368,66.806007,2.417640,0.846467,0.797514
2,1.296005e+08,3.625996e+06,44.116626,1.120321e+08,0.832181,0.852532,8.035310,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,26.495387,886.771930,1650.667839,21.298714,13.834925,66.431493,2.459251,0.845301,0.799450
3,1.323025e+08,3.587906e+06,44.022754,1.124103e+08,0.832459,0.852542,8.024653,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,26.497614,886.883230,1564.062453,21.274581,13.773402,66.359941,2.445634,0.844378,0.801325
4,1.348108e+08,3.550906e+06,43.926397,1.127652e+08,0.832380,0.851897,10.763245,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,27.337584,886.989319,1529.804524,21.318450,13.817656,66.310747,2.408201,0.843623,0.802661
5,1.372002e+08,3.512366e+06,43.880001,1.131945e+08,0.832995,0.850805,11.341796,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,27.062026,887.169899,1507.055019,21.353309,13.761380,66.172806,2.347696,0.843425,0.805550
6,1.392692e+08,3.476698e+06,43.883670,1.135587e+08,0.832661,0.849205,11.799788,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,26.037094,887.401648,1507.000310,21.640954,13.621200,64.854756,2.309168,0.843044,0.806512
7,1.407164e+08,3.438025e+06,43.849398,1.137031e+08,0.832383,0.847056,12.093705,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,25.992461,887.544811,1513.172948,21.797552,13.512417,63.666484,2.318908,0.842881,0.807805
8,1.416229e+08,3.431106e+06,43.828558,1.137585e+08,0.834584,0.854633,12.238790,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,27.482499,887.589041,1508.330179,21.640241,13.367470,63.456555,2.339247,0.836089,0.813684
9,1.417351e+08,3.426736e+06,43.802546,1.136558e+08,0.833455,0.851963,12.085632,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,27.991365,887.652320,1501.483221,21.558960,13.478044,64.168678,2.297251,0.836097,0.812928


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      991.263000
1     1167.604000
2     1277.890000
3     1261.667000
4     1152.069000
5      979.998857
6      811.194686
7      638.818457
8      584.646000
9      571.130000
10     637.028000
11     662.625000
12     789.947000
13     786.865000
14            NaN
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,-1.772344,1.305688,1.394520,-1.771485,-1.481198,0.073029,-1.382842,1.204752,-0.990234,0.166091,...,0.984445,-0.389874,-0.480815,2.969931,-0.431194,1.774968,1.163601,0.406482,1.415122,-1.596560
1,-1.628397,1.445977,1.617168,-1.721807,-0.881748,0.527711,-1.367272,0.820657,-0.546688,0.428911,...,-0.051710,-0.353331,-1.201863,0.970958,-0.308465,1.786791,1.175566,0.949406,1.092023,-1.294324
2,-1.105388,1.399843,1.432130,-1.038286,-0.418247,0.754975,-1.386932,0.275392,0.158672,0.847995,...,-0.115145,-0.637710,-1.527088,0.312786,-1.217153,0.409447,0.851180,1.526394,0.825602,-0.999913
3,-0.633732,0.904674,0.734968,-0.576169,-0.088519,0.758903,-1.392822,-0.132596,0.824346,1.230291,...,0.633403,-0.636048,-1.190571,-0.180956,-1.370845,0.139961,0.789204,1.337570,0.614654,-0.714710
4,-0.195890,0.423679,0.019359,-0.142567,-0.182504,0.511487,0.120653,-0.104860,1.148829,1.382743,...,-0.646476,-0.009027,-0.869813,-0.376263,-1.091464,0.333806,0.746595,0.818513,0.442192,-0.511457
5,0.221192,-0.077344,-0.325210,0.381968,0.546412,0.092602,0.440388,0.657046,0.830615,1.112297,...,-0.757614,-0.214725,-0.323831,-0.505959,-0.869457,0.087303,0.627117,-0.020457,0.396927,-0.072147
6,0.582347,-0.541036,-0.297957,0.827000,0.150865,-0.521032,0.693496,1.615453,-0.215960,0.282052,...,0.345772,-0.979817,0.376858,-0.506271,0.962428,-0.526723,-0.514518,-0.554697,0.309796,0.074238
7,0.834959,-1.043781,-0.552484,1.003356,-0.178876,-1.345643,0.855928,0.558230,-0.940033,-0.632425,...,1.298413,-1.013134,0.809710,-0.471080,1.959734,-1.003221,-1.543744,-0.419639,0.272544,0.270845
8,0.993196,-1.133735,-0.707261,1.071087,2.429074,1.561122,0.936109,-0.835563,-1.089341,-1.093812,...,1.596124,0.099149,0.943440,-0.498689,0.957885,-1.638128,-1.725576,-0.137615,-1.279978,1.165081
9,1.012775,-1.190536,-0.900442,0.945623,1.091262,0.536962,0.851467,-1.495939,-1.153056,-1.335633,...,-0.048438,0.479007,1.134763,-0.537724,0.440241,-1.153784,-1.108767,-0.719942,-1.278138,1.050104


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
# validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
# validation_input = dfToInputRNN(validation_input)
# validation_input

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      991.263000
1     1167.604000
2     1277.890000
3     1261.667000
4     1152.069000
5      979.998857
6      811.194686
7      638.818457
8      584.646000
9      571.130000
10     637.028000
11     662.625000
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    662.625
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
12,0.37302,-0.228675,-1.817412,-0.320956,-2.605142,-2.578139,0.722486,1.108405,2.635823,-0.189959,...,-2.260447,2.753929,0.526707,-0.506307,-0.465998,-0.789616,-0.276498,-1.706569,-1.348539,0.91254


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.3730197 , -0.2286745 , -1.81741216, -0.32095639,
         -2.60514151, -2.5781393 ,  0.72248569,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          2.75392944,  0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    789.947
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [19]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2017756286, 3646837673, 156085323, 3854769065, 2433791929, 2697540357, 266428525, 1894976094, 2623468453, 2513906382, 173822021, 574482795, 1808262024, 568236292, 1341220215, 303748286, 4193746083, 2148116305, 294520188, 1823210225, 1032676814, 509855450, 136379739, 2401590262, 3347508835, 3898054972, 800857826, 4285711222, 3629427885, 1132924315, 3083214711, 2267350266, 4266407942, 2838038181, 125326488, 3059610736, 3617880288, 3136695068, 1048922135, 4058405466, 3764319462, 1938512031, 1076713907, 895272661, 2316724233, 2353873494, 2774264892, 1000759579, 1051905223, 858475740, 2897124512, 2635841770, 1853093056, 904099303, 3397412540, 1072317237, 1876377708, 2100248206, 1351872974, 609425589, 1760665586, 1019811693, 641334597, 2813388193, 3850711923, 3183920667, 1731766361, 2514155249, 275338319, 2808338587, 2310400521, 1141472769, 1552783502, 4161352968, 1087309679, 191770195, 1810134375, 908299587, 1015057455, 3943934499, 4113503802, 3085393917, 1211282654, 4075408292, 692162778,

loss: 723.7926635742188


Step: 89 ___________________________________________
loss: 1017.9091186523438


Step: 90 ___________________________________________
loss: 1715.6751708984375


Step: 91 ___________________________________________
loss: 1344.39697265625


Step: 92 ___________________________________________
loss: 1815.4154052734375


Step: 93 ___________________________________________
loss: 1580.6871337890625


Step: 94 ___________________________________________
loss: 2407.1328125


Step: 95 ___________________________________________
loss: 1942.16845703125


Step: 96 ___________________________________________
loss: 2027.39111328125


Step: 97 ___________________________________________
loss: 956.8023071289062


Step: 98 ___________________________________________
loss: 1223.4036865234375


Step: 99 ___________________________________________
loss: 2293.04638671875


final_seed: 574482795


In [23]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 869805.3125
Epoch 2/10000
1/1 [==============================] - 0s 7ms/step - loss: 851059.1875
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 769613.6875
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 304868.6250
Epoch 5/10000
1/1 [==============================] - 0s 8ms/step - loss: 63739.4258
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 153789.6875
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 180309.0781
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 8033.7847
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 14736.8506
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 9774.4600
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 8219.8398
Epoch 12/10000
1/1 [==============================] - 0s 7ms/step - loss

1/1 [==============================] - 0s 5ms/step - loss: 7544.6763
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 5535.0859
Epoch 100/10000
1/1 [==============================] - 0s 6ms/step - loss: 7160.4712
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 5998.8574
Epoch 102/10000
1/1 [==============================] - 0s 6ms/step - loss: 9789.9619
Epoch 103/10000
1/1 [==============================] - 0s 6ms/step - loss: 9586.0654
Epoch 104/10000
1/1 [==============================] - 0s 6ms/step - loss: 5064.1675
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 7049.2144
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 6872.7007
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 8648.2822
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - loss: 3798.0281
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 8954.68

1/1 [==============================] - 0s 4ms/step - loss: 5272.8330
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 5643.6040
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 6976.1245
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 9645.4912
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 4641.6797
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 3738.3301
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 5281.8662
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 4480.8745
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 10322.9873
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 6624.0962
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 7252.9224
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 6951.

1/1 [==============================] - 0s 4ms/step - loss: 8956.1631
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 6802.4126
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 12858.5654
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 9510.3154
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 5315.6733
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 8266.8174
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 5132.1572
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 4366.1929
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 6674.9819
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 5222.2378
Epoch 300/10000
1/1 [==============================] - 0s 5ms/step - loss: 7916.5156
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 10544

1/1 [==============================] - 0s 5ms/step - loss: 10066.3760
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 11666.5439
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 6832.2876
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 4723.2422
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 8511.6729
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 7146.4629
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - loss: 5090.4170
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 5884.7710
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 7737.5522
Epoch 395/10000
1/1 [==============================] - 0s 5ms/step - loss: 7076.5679
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 6555.1987
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 6481

...

1/1 [==============================] - 0s 4ms/step - loss: 5253.0581
Epoch 2198/10000
1/1 [==============================] - 0s 4ms/step - loss: 10382.2100
Epoch 2199/10000
1/1 [==============================] - 0s 5ms/step - loss: 3258.3777
Epoch 2200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2867.1653
Epoch 2201/10000
1/1 [==============================] - 0s 4ms/step - loss: 5368.3853
Epoch 2202/10000
1/1 [==============================] - 0s 6ms/step - loss: 3885.9705
Epoch 2203/10000
1/1 [==============================] - 0s 4ms/step - loss: 2647.9580
Epoch 2204/10000
1/1 [==============================] - 0s 4ms/step - loss: 5334.4692
Epoch 2205/10000
1/1 [==============================] - 0s 6ms/step - loss: 3163.8567
Epoch 2206/10000
1/1 [==============================] - 0s 5ms/step - loss: 3172.6475
Epoch 2207/10000
1/1 [==============================] - 0s 7ms/step - loss: 6449.0493
Epoch 2208/10000
1/1 [==============================] - 0s 6ms/step - 

1/1 [==============================] - 0s 7ms/step - loss: 7020.7114
Epoch 2293/10000
1/1 [==============================] - 0s 6ms/step - loss: 5713.3047
Epoch 2294/10000
1/1 [==============================] - 0s 6ms/step - loss: 2909.1003
Epoch 2295/10000
1/1 [==============================] - 0s 6ms/step - loss: 3599.9180
Epoch 2296/10000
1/1 [==============================] - 0s 7ms/step - loss: 3940.6589
Epoch 2297/10000
1/1 [==============================] - 0s 6ms/step - loss: 5392.4941
Epoch 2298/10000
1/1 [==============================] - 0s 6ms/step - loss: 4361.0903
Epoch 2299/10000
1/1 [==============================] - 0s 5ms/step - loss: 8462.6260
Epoch 2300/10000
1/1 [==============================] - 0s 5ms/step - loss: 4157.1802
Epoch 2301/10000
1/1 [==============================] - 0s 5ms/step - loss: 8111.7583
Epoch 2302/10000
1/1 [==============================] - 0s 4ms/step - loss: 6426.7202
Epoch 2303/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 2235.2595
Epoch 2388/10000
1/1 [==============================] - 0s 6ms/step - loss: 3135.2273
Epoch 2389/10000
1/1 [==============================] - 0s 5ms/step - loss: 2218.8523
Epoch 2390/10000
1/1 [==============================] - 0s 5ms/step - loss: 968.3823
Epoch 2391/10000
1/1 [==============================] - 0s 5ms/step - loss: 7555.3208
Epoch 2392/10000
1/1 [==============================] - 0s 4ms/step - loss: 3652.3079
Epoch 2393/10000
1/1 [==============================] - 0s 4ms/step - loss: 6752.6060
Epoch 2394/10000
1/1 [==============================] - 0s 6ms/step - loss: 9406.5156
Epoch 2395/10000
1/1 [==============================] - 0s 5ms/step - loss: 4265.8350
Epoch 2396/10000
1/1 [==============================] - 0s 5ms/step - loss: 3182.7805
Epoch 2397/10000
1/1 [==============================] - 0s 5ms/step - loss: 3569.7087
Epoch 2398/10000
1/1 [==============================] - 0s 5ms/step - lo

1/1 [==============================] - 0s 5ms/step - loss: 6014.9360
Epoch 2483/10000
1/1 [==============================] - 0s 4ms/step - loss: 2885.1208
Epoch 2484/10000
1/1 [==============================] - 0s 4ms/step - loss: 2135.0322
Epoch 2485/10000
1/1 [==============================] - 0s 5ms/step - loss: 5796.1191
Epoch 2486/10000
1/1 [==============================] - 0s 4ms/step - loss: 3213.4597
Epoch 2487/10000
1/1 [==============================] - 0s 4ms/step - loss: 4157.8291
Epoch 2488/10000
1/1 [==============================] - 0s 4ms/step - loss: 5951.0474
Epoch 2489/10000
1/1 [==============================] - 0s 5ms/step - loss: 5632.0488
Epoch 2490/10000
1/1 [==============================] - 0s 4ms/step - loss: 7532.5156
Epoch 2491/10000
1/1 [==============================] - 0s 4ms/step - loss: 3994.7668
Epoch 2492/10000
1/1 [==============================] - 0s 5ms/step - loss: 6145.9985
Epoch 2493/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 3256.0691
Epoch 2578/10000
1/1 [==============================] - 0s 4ms/step - loss: 6126.7407
Epoch 2579/10000
1/1 [==============================] - 0s 5ms/step - loss: 6691.9829
Epoch 2580/10000
1/1 [==============================] - 0s 6ms/step - loss: 4022.7976
Epoch 2581/10000
1/1 [==============================] - 0s 5ms/step - loss: 5010.5742
Epoch 2582/10000
1/1 [==============================] - 0s 5ms/step - loss: 2258.5852
Epoch 2583/10000
1/1 [==============================] - 0s 5ms/step - loss: 4817.0308
Epoch 2584/10000
1/1 [==============================] - 0s 5ms/step - loss: 6251.9473
Epoch 2585/10000
1/1 [==============================] - 0s 8ms/step - loss: 4494.7612
Epoch 2586/10000
1/1 [==============================] - 0s 5ms/step - loss: 3750.5703
Epoch 2587/10000
1/1 [==============================] - 0s 6ms/step - loss: 3820.4922
Epoch 2588/10000
1/1 [==============================] - 0s 5ms/step - l

In [24]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 599ms/step
[789.947](test_target) - [[633.8337]](prediction) = 156.11332080078125


In [25]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.197625056871893

In [26]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [27]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,-0.943236,-1.338597,-0.891494,-0.780055,-1.273491,-1.336345,-0.451783,1.148205,-1.122043,-1.121418,...,1.412346,0.557520,1.347425,1.373641,0.549175,0.697943,0.666894,-1.212140,1.262053,-1.230064
1,-0.440922,1.064416,1.396499,-0.631558,0.104144,0.267379,1.386460,0.140893,-0.184474,-0.185492,...,-0.643250,0.846797,-0.301781,-0.395561,0.854043,0.716231,0.746571,-0.024832,-0.078381,0.010708
2,1.384158,0.274181,-0.505004,1.411612,1.169347,1.068967,-0.934677,-1.289098,1.306518,1.306910,...,-0.769096,-1.404317,-1.045645,-0.978080,-1.403217,-1.414174,-1.413465,1.236972,-1.183672,1.219356


train_target:


,Distrito Federal - Consumo de Cimento (t)
0,991.263
1,1167.604
2,1277.890


test_input:


...

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
12,0.37302,-0.228675,-1.817412,-0.320956,-2.605142,-2.578139,0.722486,1.108405,2.635823,-0.189959,...,-2.260447,2.753929,0.526707,-0.506307,-0.465998,-0.789616,-0.276498,-1.706569,-1.348539,0.91254


test_target:


,Distrito Federal - Consumo de Cimento (t)
12,789.947


1/1 [==============================] - 1s 555ms/step
Error: 156.11332080078125




[1273.4349365234375,
 1234.5130615234375,
 1091.2630615234375,
 915.0855712890625,
 787.0631713867188,
 632.5889282226562,
 588.4298095703125,
 591.7426147460938,
 645.7329711914062,
 633.8336791992188]

In [28]:
display(targets)
display(predictions)

[1261.667,
 1152.069,
 979.9988571428572,
 811.1946857142857,
 638.8184571428571,
 584.646,
 571.13,
 637.028,
 662.625,
 789.947]

[1273.4349365234375,
 1234.5130615234375,
 1091.2630615234375,
 915.0855712890625,
 787.0631713867188,
 632.5889282226562,
 588.4298095703125,
 591.7426147460938,
 645.7329711914062,
 633.8336791992188]

In [29]:
mae = mean_absolute_error(predictions, targets)
mae

74.1145274902344

In [30]:
porcentage = mae/np.mean(targets)
porcentage

0.09162243957471093